In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms


print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter
import timm

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner, Backbone
from lark.ops import Sig2Spec, MixedSig2Spec

from tqdm.auto import tqdm
from lark.ops import f1



1.8.1
0.8.0a0+e4e171a


# attention

In [3]:
rank = 0

In [4]:
cfg = Config(
        sites=['COR', 'SSW'],
        use_neptune=False,
        n_epochs=10,
        bs=64,
        n_samples_per_label=100,
        lr=1e-3,
        model='tf_efficientnet_b0_ns',
        scheduler='torch.optim.lr_scheduler.CosineAnnealingLR',
        use_pink_noise=0.1,
        use_recorded_noise=0.2,
        use_overlays=True,
        apply_filter=0.1,
        seed=231,
        n_workers=6,
    )

In [5]:
def make_model(cfg: Config, rank: int):
    prep = MixedSig2Spec(cfg, rank)
    backbone = Backbone('tf_efficientnet_b0_ns', pretrained=True)
    embedding_size = 512
    neck = torch.nn.Sequential(
        torch.nn.Dropout(0.3),
        torch.nn.Linear(in_features=backbone.out_features, out_features=embedding_size, bias=True),
        torch.nn.BatchNorm1d(embedding_size),
        torch.nn.PReLU()
    )
    head = torch.nn.Linear(in_features=embedding_size, out_features=cfg.n_labels)
    model = torch.nn.Sequential(prep, backbone, neck, head)
    return model.to(rank)

In [6]:
model = make_model(cfg, rank)

In [12]:
lrn = Learner("tf_efficientnet_b0_ns", cfg, rank, model)

tf_efficientnet_b0_ns-20210619-162543:0


# ogg 2 np: bad idea
takes up a massive amount of disk space

In [46]:
import librosa
import glob
import pathlib


In [53]:
input_folder = "data/birdclef-2021/train_short_audio"
output_folder = "data/birdclef-2021/train_short_audio.npy"


In [54]:
input_files = glob.glob(f"{input_folder}/*/*.ogg")

In [60]:
f = input_files[0]
f

'data/birdclef-2021/train_short_audio/amepip/XC452810.ogg'

In [56]:
def ogg2np(fname, output_folder):
    signal, _ = librosa.load(path=fname, sr=32000, mono=True)
    tokens = fname.split('/')
    basename = tokens[-1].split('.')[-2]
    bird_name = tokens[-2]
    output_dname = f"{output_folder}/{bird_name}"
    pathlib.Path(output_dname).mkdir(parents=True, exist_ok=True)
    output_fname = f"{output_dname}/{basename}"
    np.save(output_fname, signal)

In [59]:
# [ogg2np(f, output_folder) for f in input_files]

In [29]:
len(signal)

346016